In [63]:
import pandas as pd
import numpy as np
import datetime as dt
import requests as rq
import pytz

In [65]:
def short_format(num):
    if num > -10000 and num < 10000:
        return '{:,.4g}'.format(num)
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

In [67]:
def get_us_data(date=None, field=None):
    """
    Returns a JSON object with the following notable fields:
    positive, negative, death, total (number of tests), recovered
    hospitalizedCurrently, inIcuCurrently, onVentilatorCurrently
    deathIncrease, hospitalizedIncrease, positiveIncrease, negativeIncrease, totalTestResultsIncrease
    """
    j = rq.get("https://covidtracking.com/api/v1/us/{}.json".format(date.strftime('%Y%m%d') if date else "current")).json()
    j = j if date else j[0]
    return j[field] if field else j

In [68]:
def get_state_data(state, date=None, field=None):
  """
  Returns a JSON object with the following notable fields:
    positive, negative, death, total (number of tests), recovered
    hospitalizedCurrently, inIcuCurrently, onVentilatorCurrently
    deathIncrease, hospitalizedIncrease, positiveIncrease, negativeIncrease, totalTestResultsIncrease
  """
  j = rq.get("https://covidtracking.com/api/v1/states/{}/{}.json".format(state, date.strftime('%Y%m%d') if date else "current")).json()
  return j[field] if field else j

In [69]:
def percent_change(old, new):
  return (new - old) / old * 100

In [70]:
def process_data(states):
    date = states[(states['Region'] == 'San Francisco') & (states['Date']>'2020-03-10')]['Date']
    santaclara = states[(states['Region'] == 'Santa Clara') & (states['Date']>'2020-03-10')]['Estimated Effective R']
    alam = states[(states['Region'] == 'Alameda') & (states['Date']>'2020-03-10')]['Estimated Effective R']
    cc = states[(states['Region'] == 'Contra Costa') & (states['Date']>'2020-03-10')]['Estimated Effective R']
    marin = states[(states['Region'] == 'Marin') & (states['Date']>'2020-03-10')]['Estimated Effective R']
    smateo = states[(states['Region'] == 'San Mateo') & (states['Date']>'2020-03-10')]['Estimated Effective R']
    sf = states[(states['Region'] == 'San Francisco') & (states['Date']>'2020-03-10')]['Estimated Effective R']
    deaths = pd.DataFrame()

    deaths['Date'] = date.iloc[:].values
    deaths['Santa Clara'] = santaclara.iloc[:].values
    deaths['SF'] = sf.iloc[:].values
    deaths['Alameda'] = alam.iloc[:].values
    deaths['Contra Costa'] = cc.iloc[:].values
    deaths['Marin'] = marin.iloc[:].values
    deaths['San Mateo'] = smateo.iloc[:].values
    return deaths

In [71]:
def get_updated_data(df, di):
    return None

In [72]:
CSV_URL = "https://ca-covid-r.info/ca_daily_cases_and_r.csv"
df = pd.read_csv(CSV_URL)
df

,county,totalcountconfirmed,totalcountdeaths,newcountconfirmed,newcountdeaths,date
0,Santa Clara,151.0,6.0,151,6,2020-03-18
1,Santa Clara,183.0,8.0,32,2,2020-03-19
2,Santa Clara,246.0,8.0,63,0,2020-03-20
3,Santa Clara,269.0,10.0,23,2,2020-03-21
4,Santa Clara,284.0,13.0,15,3,2020-03-22
...,...,...,...,...,...,...
7440,Yolo,1034.0,30.0,32,1,2020-07-16
7441,Yolo,1055.0,31.0,21,1,2020-07-17
7442,Yolo,1077.0,31.0,22,0,2020-07-18
7443,Yolo,1127.0,31.0,50,0,2020-07-19


In [73]:
clean_df = process_data(df)
clean_df

,Date,Santa Clara,SF,Alameda,Contra Costa,Marin,San Mateo
0,2020-03-18,6.0,0.0,0.0,0.0,0.0,1.0
1,2020-03-19,8.0,0.0,0.0,0.0,0.0,1.0
2,2020-03-20,8.0,0.0,0.0,1.0,0.0,1.0
3,2020-03-21,10.0,0.0,0.0,1.0,0.0,1.0
4,2020-03-22,13.0,0.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...
120,2020-07-16,176.0,51.0,154.0,93.0,29.0,114.0
121,2020-07-17,177.0,52.0,161.0,96.0,29.0,114.0
122,2020-07-18,177.0,52.0,162.0,98.0,30.0,114.0
123,2020-07-19,177.0,52.0,162.0,98.0,30.0,114.0


In [74]:
get_updated_data(clean_df, None)

{'smart_tiles': [{},
  {},
  {'figure': '61', 'subheader': 'On 7/21'},
  {'figure': '1,029', 'subheader': 'On 7/21'}]}